<a href="https://colab.research.google.com/github/crxstxan00/assassin_screed/blob/main/XatBotclozano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask-cors
!pip install pyngrok
import requests
from bs4 import BeautifulSoup
from google import genai
from google.colab import userdata
from flask import Flask, request, jsonify
import threading
from pyngrok import ngrok
# ==============================
# CONFIGURACIÓN DE LA API
# ==============================
GOOGLE_API_KEY = userdata.get("GeminiAPI")
if not GOOGLE_API_KEY:
    print("⚠️ No se ha encontrado la clave API. Configúrala antes de ejecutar el programa.")
    exit()

client = genai.Client(api_key=GOOGLE_API_KEY)

# ==============================
# CORS
# ==============================
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})  # Permite CORS para todas las rutas y dominios


# ==============================
# ngrok
# ==============================
# Sustituye 'TU_NGROK_AUTH_TOKEN' por el token que copiaste desde el panel de ngrok
ngrok.set_auth_token('2uIxOjS2M0jsrm08HYkpYkThyjb_7xiRLBvPj7A7H86UKQgvN')

# ==============================
# LIBERAR PUERTO 5000
# ==============================
!kill -9 $(lsof -t -i:5000)

# ==============================
# INICIAR FLASK
# ==============================
app = Flask(__name__)

# Exponer el puerto en ngrok
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

# ==============================
# FUNCIÓN DE SCRAPING EN UNA ÚNICA PÁGINA
# ==============================
def obtener_info_web(url):
    try:
        respuesta = requests.get(url, timeout=1)
        respuesta.raise_for_status()
        soup = BeautifulSoup(respuesta.text, "html.parser")

        # Extraer información de la página actual
        titulo = soup.title.string if soup.title else "Sin título"
        parrafos = [p.text for p in soup.find_all("p")]
        contenido = f"Título: {titulo}\nContenido: {' '.join(parrafos[:50])}"

        return contenido
    except Exception as e:
        return f"Error al obtener datos de la web: {e}"

# ==============================
# INSTRUCCIONES DEL SISTEMA
# ==============================
system_instruction = """
Eres un asistente de inteligencia artificial que usa información en tiempo real de la web.
Proporciona respuestas claras y basadas en los datos extraídos, omite todo sobre los gustos.
"""

chat = client.chats.create(
    model="gemini-2.0-flash",
    config=genai.types.GenerateContentConfig(
        system_instruction=system_instruction,
        temperature=0.7,
        max_output_tokens=200
    )
)

# ==============================
# RUTA DEL CHATBOT
# ==============================
@app.route('/chatbot', methods=['POST'])
def chatbot():
    user_message = request.json.get('message')  # Obtener el mensaje del usuario

    if not user_message:
        return jsonify({"error": "No se ha proporcionado un mensaje."}), 400

    try:
        # Obtener información de la página web
        info_web = obtener_info_web("https://clozano.inscastellbisbal.net/ipop/")

        # Enviar el mensaje al modelo y obtener una respuesta
        respuesta = chat.send_message(f"{info_web}\n\nPregunta: {user_message}")

        # Devolver la respuesta del modelo
        return jsonify({"response": respuesta.text.strip()})

    except Exception as e:
        return jsonify({"error": f"Error en la comunicación con Gemini: {e}"}), 500


@app.after_request
def add_cors_headers(response):
    response.headers["Access-Control-Allow-Origin"] = "*"
    response.headers["Access-Control-Allow-Methods"] = "GET, POST, OPTIONS"
    response.headers["Access-Control-Allow-Headers"] = "Content-Type"
    return response

# ==============================
# INICIAR EL SERVIDOR
# ==============================
if __name__ == '__main__':
  app.run(port=5000)

kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]
Public URL: NgrokTunnel: "https://4314-104-154-69-130.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [18/Mar/2025 13:02:44] "OPTIONS /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Mar/2025 13:02:47] "POST /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Mar/2025 13:02:58] "OPTIONS /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Mar/2025 13:03:01] "POST /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Mar/2025 13:03:18] "OPTIONS /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Mar/2025 13:03:20] "POST /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Mar/2025 13:03:55] "OPTIONS /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Mar/2025 13:03:57] "POST /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Mar/2025 13:04:09] "OPTIONS /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Mar/2025 13